In [ ]:
# input all the fasta files into assembler per species. make flags to indicate which databases are contributing to outcome. 

# Strains that have only Contigs

In [76]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rc
import pandas as pd
import sqlite3

cnx = sqlite3.connect(r'/Users/aaronkarlsberg/Desktop/Djoin/data/refSeqFungiStats.db')
strain_with_contigs_only = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)

print("strain_with_contigs_only", strain_with_contigs_only.size)


# canu requires approximate genome size as input: for now we take average contig length and multiply by the total number of contigs within a single fasta file. The mazimum between fasta files for a species will serve as the approximate genome length.
def generate_canu_command_file_for_strain_contigs(filepaths, contributing_databases, approx_genome_length, strainTaxid):
    approx_genome_length = str(approx_genome_length)
    files = []
    for file in filepaths:
        files.append(file.strip())
    files = ' '.join(files)
    distinct_databases = set(contributing_databases)
    distinct_database_count = str(len(distinct_databases))
    contributing_databases = '_'.join(contributing_databases)
    shell_script= open("/Users/aaronkarlsberg/Desktop/Djoin/canu_shell_scripts/contig_only_strain_assembly_canu_shell_command_script.sh","a+")
    canu_command = "/u/home/a/akarlsbe/scratch/canu-1.8/*/bin/canu -d /u/home/a/akarlsbe/scratch/strain_contig_only_assemblies/"+ strainTaxid +" -p "+distinct_database_count+"_contributing_databases_"+ contributing_databases +" genomeSize="+ approx_genome_length +" -nanopore-raw " + files +" useGrid=false stopOnLowCoverage=2\n"
    shell_script.write(canu_command)
    shell_script.close()
    return

for index, row in strain_with_contigs_only.iterrows():
    strain_tax_id = str(row["STRAINTAXID"])
    strain_filepaths_query = pd.read_sql_query("SELECT FILEPATH, DBNAME, contig_count, avg_length_contig FROM speciesdb where STRAINTAXID = "+ strain_tax_id +" AND chromosome_count = 0 AND contig_count > 0 ", cnx)
    strain_filepaths_list = []
    strain_databases = []
    contig_counts = []
    avg_contig_lengths = []
    for indexL, rowL in strain_filepaths_query.iterrows():
        strain_filepaths_list.append(rowL["FILEPATH"])
        strain_databases.append(rowL["DBNAME"])
        contig_counts.append(rowL["contig_count"])
        avg_contig_lengths.append(rowL["avg_length_contig"])
        genome_lengths = []
        for i in range(0,len(contig_counts)):
            genome_lengths.append(contig_counts[i] * avg_contig_lengths[i])
        approx_genome_length = max(genome_lengths)
    generate_canu_command_file_for_strain_contigs(strain_filepaths_list, strain_databases, approx_genome_length, strain_tax_id)  
    
print(strain_filepaths_list, strain_databases, contig_counts,avg_contig_lengths)


strain_with_contigs_only 211
['/u/home/a/akarlsbe/scratch/fungi/ENSEMBLE/Aspergillus_nomius_nrrl_13137.ASM120477v1.dna.toplevel.fa\n', '/u/home/a/akarlsbe/scratch/fungi/1K/Aspnom13137_1_AssemblyScaffolds_Repeatmasked.fasta\n'] ['ENSEMBLE', '1K'] [1118, 1118] [32863, 32786]


# Strains that have only Chromosomes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rc
import pandas as pd
import sqlite3

cnx = sqlite3.connect(r'Djoin/data/refSeqFungiStats.db')
strain_with_chromosomes_only =pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)



# Strains that have both Chromosomes and Contigs within the same fasta file: "Mixed Sequence Files"

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rc
import pandas as pd
import sqlite3

cnx = sqlite3.connect(r'Djoin/data/refSeqFungiStats.db')
chrom_and_contig_mixed =pd.read_sql_query("SELECT * FROM speciesdb where chromosome_count > 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)



# Strains that have only chromosomes and only contigs in seperate databases

In [36]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rc
import pandas as pd
import sqlite3

cnx = sqlite3.connect(r'Djoin/data/refSeqFungiStats.db')


strain_with_contigs_in_onedb_chrom_in_other = []

strain_with_contigs_only = pd.read_sql_query("SELECT * FROM SPECIESDB where chromosome_count > 0 AND contig_count = 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)

strain_with_chromosomes_only = pd.read_sql_query("SELECT * FROM SPECIESDB where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)


# for indexCont, rowCont in strain_with_contigs_only.iterrows():
#     for indexChrom, rowChrom in strain_with_chromosomes_only.iterrows():
#         if rowCont["STRAINTAXID"] == rowChrom["STRAINTAXID"]:
#             strain_with_contigs_in_onedb_chrom_in_other.append(rowCont["STRAINTAXID"])

# # Eliminate dublicates
# strain_with_contigs_in_onedb_chrom_in_other = list(dict.fromkeys(strain_with_contigs_in_onedb_chrom_in_other))

# print(strain_with_contigs_in_onedb_chrom_in_other)

# output of the code that is commented out above. This is to avoid running nested for loop more than once. 
strain_with_contigs_in_onedb_chrom_in_other = [227321, 237561, 1071382, 1230383, 871575, 665079, 284813, 425011]



for straintaxid in strain_with_contigs_in_onedb_chrom_in_other:
    pd.read_sql_query("select * from SPECIESDB where straintaxid = "+ str(straintaxid), cnx)









# Species that have only chromosomes and only contigs in seperate databases

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rc
import pandas as pd
import sqlite3

cnx = sqlite3.connect(r'Djoin/data/refSeqFungiStats.db')

species_with_contigs_in_onedb_chrom_in_other = []

species_with_contigs_only = pd.read_sql_query("SELECT * FROM speciesdb where chromosome_count > 0 AND contig_count = 0 AND SPECIESTAXID !=0 Group by SPECIESTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)

species_with_chromosomes_only = pd.read_sql_query("SELECT * FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND SPECIESTAXID !=0 Group by SPECIESNTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx)


# for indexCont, rowCont in species_with_contigs_only.iterrows():
#     for indexChrom, rowChrom in species_with_chromosomes_only.iterrows():
#         if rowCont["SPECIESTAXID"] == rowChrom["SPECIESTAXID"]:
#             species_with_contigs_in_onedb_chrom_in_other.append(rowCont["SPECIESTAXID"])

# # Eliminate dublicates
# species_with_contigs_in_onedb_chrom_in_other = list(dict.fromkeys(species_with_contigs_in_onedb_chrom_in_other))

# print(species_with_contigs_in_onedb_chrom_in_other)

# output of the code that is commented out above. This is to avoid running nested for loop more than once. 
species_with_contigs_in_onedb_chrom_in_other = [746128, 162425, 40559, 5476, 5478, 37769, 432096, 76777, 1005962, 5076, 4932, 5180, 4950, 27337, 4952, 4956, 6035, 109760, 5346, 5062, 61459, 263815, 101852, 1033840, 563466, 29908, 68825, 40302, 498019, 1664694, 42068, 4754, 398673, 254056, 856822, 1367422, 280754, 80884, 5619, 104669, 1209926, 4909, 5127, 5061, 5480]


for speciestaxid in species_with_contigs_in_onedb_chrom_in_other:
    pd.read_sql_query("select * from speciesdb where speciestaxid = "+ str(speciestaxid), cnx)




# Use cases to revisit:

In [ ]:
# 72 instances in which the fungi_stats code did not characterize the sequence types contained in the fasta files. This is evident where chromosome and contig counts are zero.

unclassified_sequences = pd.read_sql_query("SELECT * FROM speciesdb where chromosome_count = 0 AND contig_count = 0 ", cnx).size



In [48]:
strain_with_contigs_only1 = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0", cnx).size
print(strain_with_contigs_only1)

strain_with_contigs_only = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 0", cnx).size
print(strain_with_contigs_only)

strain_with_contigs_only2 = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) = 1", cnx).size
print(strain_with_contigs_only2)

strain_with_contigs_only4 = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(DISTINCT DBNAME) > 1", cnx).size
print(strain_with_contigs_only4)


strain_with_contigs_only3 = pd.read_sql_query("SELECT STRAINTAXID FROM speciesdb where chromosome_count = 0 AND contig_count > 0 AND STRAINTAXID !=0 Group by STRAINTAXID HAVING COUNT(STRAINTAXID) > 1", cnx).size

print(strain_with_contigs_only3)


782
541
330
211
213


In [81]:
strain_with_contigs_only5 = pd.read_sql_query("SELECT STRAINTAXID,DBNAME FROM speciesdb where STRAINTAXID !=0 Group by STRAINTAXID, DBNAME HAVING COUNT(*) > 1", cnx)


print(strain_with_contigs_only5)





    STRAINTAXID    DBNAME
0         56615  ENSEMBLE
1         62688        1K
2        227321  ENSEMBLE
3        237561  ENSEMBLE
4        644223  ENSEMBLE
5        665079  ENSEMBLE
6        683840        1K
7        698492  ENSEMBLE
8        701091        1K
9        764103  ENSEMBLE
10      1016881        1K
11      1230383  ENSEMBLE
